## Simpler code with basics of semi-mechanistic approach
Again, thinking about this equation in Faria, et al:
$\\i_{s,t} = (1-\frac{n_{s,t}}{N})R_{s,t}\sum_{\tau<t} i_{s,\tau}g_{t-\tau}$

And again reducing the complexity of this by ignoring strains, I'll consider:
$\\i_t = (1-\frac{n_t}{N})R_t\sum_{\tau<t} i_{\tau}g_{t-\tau}$

Now aiming to include susceptible depletion and move to:
$\\i_t = R_t\sum_{\tau<t} i_\tau g_{t-\tau}$

In [ ]:
from typing import Dict
from scipy.stats import gamma
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'

Get some sort of distribution we can sensibly use for the generation time

In [ ]:
def get_gamma_params_from_mean_sd(req_mean: float, req_sd: float) -> Dict[str, float]:
    var = req_sd ** 2.0
    scale = var / req_mean
    a = req_mean / scale
    return {'a': a, 'scale': scale}

In [ ]:
def update_inc(working_vals, time, gen_time_dens, r0):
    return (working_vals[:time] * gen_time_dens[time-1::-1]).sum() * r0

In [ ]:
# Model parameters
seed = 1.0
r0 = 2.0
gen_time_sd = 1.5
gen_time_mean = 5.0

This is going to just keep going up exponentially, of course, because $R_{0} > 1$ and there is no susceptible depletion

In [ ]:
gamma_params = get_gamma_params_from_mean_sd(gen_time_mean, gen_time_sd)
times = 20
inc = np.zeros(times)
inc[0] = seed
gen_time_densities = gamma.pdf(range(times), **gamma_params)
for t in range(1, times):
    inc[t] = update_inc(inc, t, gen_time_densities, r0)

In [ ]:
pd.Series(inc).plot(labels={'index': 'day', 'value': 'incidence'})

Already some interesting phenomena there, 
in that the humps are the generations of cases from the first seeding infection,
which progressively smooth into one-another with generations of cases.
That's it for now.
Next step is that I'll try to incorporate the depletion of susceptibles.